# Tutorial: Topic Modeling

## Step 1: Introduction to the Basics of Topic Modeling

Sometimes we have a corpus of text and just want to understand what topics are being discussed in the text. This is where topic modeling shines. Topic modeling is an unsupervised machine learning technique that is capable of scanning a set of documents, detecting word and phrase patterns within them, and automatically clustering word groups that best characterize a set of documents. Think about it a bit like factor analysis, but for text. To be clear, *It is not factor analysis, but it is an apt analogy in that it is looking for underlying patterns in the data.*

To start, we need to load the corpus of text that we want to analyze. In this tutorial, we will use the two datasets: the 20 newsgroups dataset and a set of article abstracts from management and entrepreneurship journals. The reason we are using the 20 newsgroups dataset is that it is a well-known dataset that is often used to demonstrate NLP techniques such as topic modeling and has so much text that you couldn't hope to examine it all manually. We'll also use the management and entrepreneurship dataset to show some shortcomings of topic modeling in smaller datasets.

Let's start by loading the 20 newsgroups dataset and examining the first few rows.

In [ ]:
# Load the 20 newsgroups dataset
from sklearn.datasets import fetch_20newsgroups
print("Getting the 20 newsgroups dataset... this may take a few minutes...")
newsgroups = fetch_20newsgroups(subset='all')
for idx in range(5):
    print(f"Document {idx}:")
    print(f"\tlabel: {newsgroups.target_names[newsgroups.target[idx]]}")
    print(f"\ttext: {newsgroups.data[idx].replace('\n', ' ')}\n")
print(f"Total number of documents: {len(newsgroups.data)}")
print(f"Number of labels: {len(newsgroups.target_names)}")


19k texts - yeah, I think you'd find it challenging to read all of those manually. Note also there are 20 labels - these are the newsgroups that the texts are from. We won't use these labels in our topic modeling, but they could be useful for analyses such as classification (which we'll get to next week). However, these labels might also provide some insight into the topics that are being discussed in the text - which we do care about.

One thing to note - just like before, these are just strings/texts. Most of the time, we'll need to do some preprocessing before we can do any analysis. Normally we'd use SpaCy, Stanza, or a more sophisticated library to do this for us, but with 18,846 texts on your laptop (which may or may not have a GPU), this could take a long while. So, we'll just use a simple function to preprocess the text. This function will crudely remove punctuation, numbers, and stopwords, and then lowercase the text. This is a very basic preprocessing step, but it should be enough for our purposes.

In [ ]:
def preprocess(text: str) -> list:
    import re
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'[^A-Za-z]', ' ', text)
    words = word_tokenize(text.lower())
    words = [word.lower() for word in words if word not in stop_words and word.isalpha()]
    return words

my_text = "This is a test of the preprocess function."
print(my_text)
print(preprocess(my_text))

OK, that worked. Now let's preprocess the *entire* corpus of text. This could take a while depending on your computer, so be patient.

In [ ]:
from pprint import pprint
preprocessed = [preprocess(text) for text in newsgroups.data]
pprint(preprocessed[0:3], indent=4, compact=True, width=100)

OK, we're ready to do some basic topic modeling. We're going to use the Tomotopy library, but you could also use Gensim or Scikit-learn. A nice thing about Tomotopy is that it has several different topic modeling algorithms, including Latent Dirichlet Allocation (LDA), DMR, and others. We'll use LDA for this part of the tutorial.

With LDA, we need to specify a number of *hyperparameters*, including 
1. `k` - number of topics - This can be anything, but since this is the "20 newsgroups" dataset, we'll specify 20 topics for the time being. 
2. `alpha` - This is the prior on the per-document topic distribution. A low value of alpha means that each document is likely to contain only a few topics, while a high value means that each document is likely to contain many topics. We'll specify a value of 0.1 for alpha.
3. `eta` - This is the prior on the per-topic word distribution. A low value of eta means that each topic is likely to contain only a few words, while a high value means that each topic is likely to contain many words. We'll specify a value of 0.01 for eta.

There are also some other algorithm parameters that we need to specify, including
1. Number of iterations we want to run the algorithm for - LDA is an iterative algorithm. If you ran just one iteration, you likely would not get convergence and the results would be garbage. On the other hand, you could run forever with diminishing returns and increased potential for overfitting. 
2. Burn-in - This is the number of iterations that we want to run before we start recording the results. This is because the first few iterations are often not very good when using Gibbs sampling where the algorithm initializes with random topics. We want to discard these early iterations.

In [ ]:
import tomotopy as tp
mdl = tp.LDAModel(k=20, alpha=0.1, eta=0.01)
for doc in preprocessed:
    mdl.add_doc(doc)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
mdl.burn_in = 100
mdl.train(1000, show_progress=True)
mdl.summary()

Look at the `<Topics>` output, these are the topics that the algorithm has identified. Each topic is a list of words that are most likely to be associated with that topic. The first words in each topic are the most likely to be associated with that topic. Technically, every word is associated with every topic, much like with exploratory factor analysis, but some words are more likely to be associated with some topics than others. Think of it a bit like EFA's factor loadings. Every indicator is associated with every factor, but some indicators are more strongly associated with some factors than others. Similar idea here, but with words and topics.

Do you notice that some of these words are not very informative? In my run (yours may be different), I have a topic: `| #5 (179446) : w p r u c` - not very informative. Part of this comes down to preprocessing, perhaps I should have eliminated one-letter words. But another part comes down to a few more algorithm parameters we didn't touch:
1. `tw` - term weighting - This is how the algorithm weights the words in the document. The default is `one` where each word is weighted equally. However, a notable alternative is `idf` where words that are common across all documents are weighted less than words that are unique to a few documents.
2. `rm_top` - remove top words - This is the number of most frequent words to remove from the analysis. The default is 0, but you could remove the top 10, 20, 50, etc. words from the analysis. This can be useful if you have a lot of words that are common across all documents and are not very informative.
3. `min_cf` - minimum collection frequency - This is the minimum number of times a word must appear in the corpus to be included in the analysis. In other words, how many times would a word have to show up in the entire corpus to be considered for inclusion in a 'topic'. Only once? 10 times? 100 times? This is what this parameter controls.
4. `min_df` - minimum document frequency - This is the minimum number of documents a word must appear in to be included in the analysis. In other words, how many documents would a word have to show up in to be considered for inclusion in a 'topic'. Only 1 document? 10 documents? 100 documents? This is what this parameter controls.

Let's see if we can get better topics by adjusting these parameters and eliminating words with two or fewer characters.

In [ ]:
preprocessed_v2 = []
for doc in preprocessed:
    preprocessed_v2.append([word for word in doc if len(word) > 2])

mdl = tp.LDAModel(tw=tp.TermWeight.IDF, k=20, alpha=0.1, eta=0.01, rm_top=10, min_cf=100, min_df=50)
for doc in preprocessed_v2:
    mdl.add_doc(doc)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
mdl.burn_in = 100
mdl.train(1000, show_progress=True)
mdl.summary()

Let's take a look at some of this output to understand what it is saying: (your output may be different)

```
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.02406081 0.02029392 0.03429637 0.00471696 0.01331376 0.06557953
|   0.0146365  0.01155585 0.02238731 0.01136403 0.02682437 0.01167304
|   0.01151419 0.08631996 0.04910115 0.01652213 0.07590692 0.01145388
|   0.00974874 0.01921702]
| eta (Dirichlet prior on the per-topic word distribution)
|  0.01
|
<Topics>
| #0 (159317) : file image jpeg window files
| #1 (104997) : god jesus bible christian christ
| #2 (150040) : scsi dos drive windows card
```

Recall that `alpha` is the per-document topic distribution - this roughly captures the sparsity of topics in each document. A low value of alpha means that each document is likely to contain only a few topics, while a high value means that each document is likely to contain many topics. Now `alpha` has a value for each of our topics. Topic 19 has an alpha of 0.009 whereas topic 13 has an alpha of 0.086. This means that topic 19 is likely to be less common across documents than topic 13.

Recall that `eta` is the per-topic word distribution - this roughly captures the sparsity of words in each topic. .01 is a low value, which means that each topic is likely to be dominated by only a few words.

The `<Topics>` output shows the top words for each topic and the number in the parentheses is the number of times words associated with that topic appears in the corpus. This is a good way to see how common each topic is across the corpus.

Looking at the results' interpretability, those topics seem a lot cleaner to me. I can actually understand what they are about. This is a good example of how tweaking the algorithm parameters can make a big difference in the results.

#0 seems to be about images, #1 seems to be about religion, and #2 seems to be about computer hardware. These are pretty clear topics. But that's human interpretability, there are also some quantitative measures that we can use to evaluate the quality of the topics. One common measure is the coherence of the topics. This is a measure of how semantically similar the words in a topic are. The higher the coherence, the (presumably) better the topic.

In [ ]:
# calculate coherence using preset
for preset in ('u_mass', 'c_uci', 'c_npmi', 'c_v'):
    coh = tp.coherence.Coherence(mdl, coherence=preset)
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence: {} ===='.format(preset))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

Generally we're looking for higher coherence values, but each metric has a different scale. For instance, u_mass is generally between -14 and 0 (where closer to zero is better), while c_v is generally between 0 and 1 (where closer to 1 is better).

Look at your coherence scores. Are they good? Bad? Are there individual topics that seem to do poorly on coherence? Look back at those topics, are they harder to interpret? 

The tuning of the hyperparameters and algorithm parameters is a bit of an art. You can't just throw a bunch of words at the algorithm and expect it to spit out perfect topics. You need to tweak the parameters to get the best results. This is where domain knowledge can be very helpful. If you know a lot about the data, you can make more informed decisions about the parameters. If you don't know much about the data, you may need to try a few different sets of parameters to see what works best.

However, a couple of traps:
1. **Overfitting the model to the data** - Tweaking the parameters too much may result in a model that explains the data you have very well, but is not very good at explaining new data. This is a common problem in machine learning, and it is something to be aware of when tuning the parameters.
2. **Neglecting human interpretability** - There is a tendency to reify coherence scores as the be-all, end-all of topic model evaluation. However, if the topics are not interpretable by humans, then the model is not very useful. It is important to balance the quantitative measures with the qualitative measures.

Now it's your turn. Try tweaking the parameters and see if you can get better/different topics. Try looking at different numbers of topics. Remember, this is a bit of an art, so don't be afraid to experiment.

In [28]:
#Your playground here

## Step 2: Visualizing LDA Topics

Sometimes it is nice to be able to visualize the topics that the algorithm has identified. One way to do this is to use a word cloud. A word cloud is a visualization that shows the frequency of words in a text. The more frequent a word is, the larger it appears in the word cloud. We're going to abuse this a little bit. Instead of providing the frequency of words in a text, we're going to provide the probability of a word being associated with a topic. The more probable a word is to be associated with a topic, the larger it will appear in the word cloud.

Let's create a word cloud for the first 2 topics. You can change 2 to any number of topics you want to visualize so long as you don't increase the number beyond what you specified for `k` in the model.

In [ ]:
#Generate a word cloud for the first 2 topics
from wordcloud import WordCloud
import matplotlib.pyplot as plt
for k in range(2):
    print('Topic', k)
    print('Words:', mdl.get_topic_words(k, top_n=10))
    plt.imshow(WordCloud(width=800, height=400).generate_from_frequencies(dict(mdl.get_topic_words(k, top_n=10))))
    plt.axis('off')
    plt.show()

That's handy, but it's a bit simplistic - it tells us one aspect of how important words are to a topic, but that's about it. Let's use pyLDAvis to visualize the topics in a more sophisticated way. pyLDAvis provides a lot more information about the topics.

In [ ]:
import pyLDAvis
import numpy as np

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency,
    start_index=1,
    sort_topics=False
)
pyLDAvis.display(prepared_data)

Click on "Next Topic" to cycle through the various topics. You can see the top words in each topic, the frequency of the words in the corpus (and in the topic). 

You can also drag the λ slider at the top right to shift the balance in the relevance metric between the term frequency and the topic-specific term frequency. This can help you see different perspectives on the topics' most salient words. 
* Closer to 1 (term probability) shows the terms that are most frequent within the topic regardless of whether they may also be frequently used in other topics as well. 
* Closer to 0 (term exclusivity) shows the terms that are most exclusive to the topic, meaning they are highly unique to the topic, while placing less emphasis on the frequency. Here you'll see that the red bars (term frequency in the topic) are a greater proportion of the blue bars (term frequency in the corpus) - indicating that the term is more unique to the topic.

Also try hovering over individual words to see how they are distributed across the topics. This can give you a sense of how unique a word is to a topic and to what extent it may be reflective of multiple topics.

Important note: Tomotopy starts numbering topics with 0, but pyLDAvis starts numbering topics with 1. So, if you want to see topic 0, you need to look at topic 1 in pyLDAvis.



## Step 3: Other Topic Modeling Algorithms

Latent Dirichlet Allocation is a workhorse of topic modeling, not unlike OLS for regression. However, there are other algorithms that can be used for topic modeling. For instance, Latent Dirichlet Allocation assumes that the topics are uncorrelated, but in reality, we know that topics are often correlated. This is where algorithms such as correlated topic models (CTM) come in. CTM is an extension of LDA that allows for correlations between topics. Let's model the 20 newsgroups dataset using CTM and see what we get.

Be prepared to wait a bit for the analysis to complete - this is more computationally intensive than LDA.

In [ ]:
mdl = tp.CTModel(tw=tp.TermWeight.IDF, k=20, smoothing_alpha=0.1, eta=0.01, rm_top=10, min_cf=100, min_df=50)
for doc in preprocessed_v2:
    mdl.add_doc(doc)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
mdl.burn_in = 100
mdl.train(1000, show_progress=True)
mdl.summary()

OK, we have new topics:

```
<Topics>
| #0 (89092) : bible law book details people
| #1 (82365) : team games hockey play henry
| #2 (80879) : church religion government people war
| #3 (73478) : gun jpeg men serdar solution
| #4 (79161) : space toronto caltech dec morality
| #5 (86004) : armenian armenia turkey method nsa
| #6 (83429) : scsi key data chip vax
| #7 (71211) : god jesus children jews jewish
```

These topics are a bit different than the LDA topics, they could probably refined and become bit more coherent if we tweaked the parameters a bit. But this is a good example of how different algorithms can produce different results. What I'm noticing is that there are 3 that seem to be about religion (one seems more about the tenets of religion (#0), one seems to be about societal institutions of which religion is one (#2), and one seems to be about individuals associated with religion (#7)). Those seem like they might be correlated. 

Let's examine this:

In [ ]:
mdl.get_correlations(topic_id=0)

```
array([ 1.        ,  0.1827854 ,  0.21705134,  0.21295892,  0.16247825,
        0.1837198 ,  0.16191892,  0.21248552,  0.13512343,  0.17616698,
        0.19299068, -0.03031998,  0.12526283,  0.08126452,  0.06282678,
        0.1061037 ,  0.08198302,  0.02917691, -0.02375286, -0.08116509],
      dtype=float32)
```
Naturally, the correlation between a topic and itself is 1. The correlation with the other two religion topics are 0.217 and 0.212, respectively. This is a little higher correlation relative to most of the others. Here too, I suspect with more coherent topics, we'd have more meaningful correlations. However, even with this untuned model, we see the emergence of correlated topics where the correlations seem to make sense.

What about if the topics are not just correlated, but hierarchical in nature? This is where hierarchical LDA (hLDA) comes in. hLDA is an extension of LDA that allows for hierarchical relationships between topics. Let's model the 20 newsgroups dataset using hLDA and see what we get. Notice that we don't specify `k` here - hLDA will determine the number of topics itself. This is one of the advantages of hLDA - it can determine the number of topics automatically. Instead, we specify depth, which is the depth of the hierarchy. We specified a depth of 3 for this analysis. We also specified `gamma` which controls the growth of the tree hierarchy. Smaller values of gamma result in a more balanced tree, while larger values result in a more complex tree.

However, let's run it for just 2,000 texts - this is a computationally intensive algorithm and running it on the full dataset would take hours. Even with this reduced dataset it will still take a while - maybe find something else to do while you wait. It takes about 25 minutes on my reasonably powerful computer.

In [ ]:
mdl = tp.HLDAModel(tw=tp.TermWeight.IDF, depth=3, alpha=0.1, eta=0.01, gamma=0.1, rm_top=10, min_cf=100, min_df=50)
for doc in preprocessed_v2[:2000]:
    mdl.add_doc(doc)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
mdl.burn_in = 100
mdl.train(1000, show_progress=True)
mdl.summary()

If your results are like mine, you got a huge number of topics (582 for me). This is the blessing and the curse of letting the algorithm identify the number of topics... you may get more (or fewer) topics than you were expecting. However, tuning the hyperparameters can help with this.

Let's take a look at the topic outputs:

```
<Topics>
| #0 (63944, 2000) : like get think time know
|   #8 (187, 11) : law christians paul god jews
|     #824 (109, 11) : day christian jesus says days
|   #9 (64, 6) : wrong right say better able
|     #214 (27, 1) : set standard tell god like
|     #278 (24, 5) : political apple cost world fact
|   #10 (33, 16) : andrew research canada however given
|     #20 (34, 10) : cmu software engineering institute high
|     #143 (23, 6) : windows program help thanks anybody
```

The indentation levels tell you about the hierarchical structure of the topics. The more indented a topic is, the more specific it is. For instance, topic 0 is the most general (root) topic, while topic 8 is a subtopic of topic 0, and topic 824 is a subtopic of topic 8. This is a nice way to see how the topics are related to each other. 

There are a bunch of other topic modeling algorithms out there, including DMR, NMF, and others. Each has its own strengths and weaknesses. Let's move on to examine what happens when we use these algorithms on a smaller dataset.

# Step 4: Topic Modeling on a Smaller Dataset

Let's go back to the management and entrepreneurship dataset. This dataset is much smaller than the 20 newsgroups dataset, with only 50 texts. Let's see what happens when we try to model this dataset using LDA.

In [ ]:
from pathlib import Path
import spacy

abstracts_txt = Path() / "data" / "abstracts.txt"
assert abstracts_txt.exists(), "Abstracts file does not exist"
with open(abstracts_txt, "r", encoding="utf-8-sig") as f:
    abstracts = f.readlines()

nlp = spacy.load("en_core_web_sm")

def preprocess(text: str) -> list:
    doc = nlp(text)
    filtered_tokens = [
        token.lemma_
        for token in doc
        if not (token.is_punct or token.is_stop or token.is_digit or token.is_space)
    ]
    return filtered_tokens


spacy_results = [preprocess(abstract) for abstract in abstracts]

print("Abstracts loaded and preprocessed.")

Normally, I'd show you what it looks like if I ran the same code as before, but I'm not going to do that here. Why? Because if I did that, there'd be no results. Why? Because the minimum document frequency is set to 50 and there are only 50 documents in the dataset. This means that no words would be included in the analysis unless they appeared in every document. This is a common problem with small datasets - you need to be careful with the parameters you choose.

So let's try it but without a minimum corpus and document frequency.

In [ ]:
mdl = tp.LDAModel(tw=tp.TermWeight.IDF, k=20, alpha=0.1, eta=0.01, rm_top=10)
for doc in spacy_results:
    mdl.add_doc(doc)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
mdl.burn_in = 100
mdl.train(1000, show_progress=True)
mdl.summary()

This doesn't seem *too* bad, but it's not great. Some of the topics are much more coherent than others. For instance, one of my topics is: `| #16 (129) : history selection method well courage` - this is not a very coherent topic. This is a common problem with small datasets - the algorithm doesn't have enough data to work with. This can sometimes be mitigated when looking for a smaller number of topics and in a corpus of texts with simpler/more straightforward topics (e.g., half of the documents are about strawberries and the other half are about bananas). However, in a corpus of texts with more complex topics, this can be a real problem.

Still, let's look at some other things we can glean from this model. For instance, we can look at the topic distribution for each document. This tells us how much of each topic is present in each document. This can be useful for understanding what topics are present in the corpus and how they are distributed across the documents.

In [ ]:
abstract_to_examine = 1

import textwrap
print("Abstract:")
print(textwrap.fill(abstracts[abstract_to_examine], width=100))
print("\nTop three topics:")
topics = mdl.docs[abstract_to_examine].get_topics(top_n=3)
print(topics,"\n")
for topic in topics:
    words = [result[0] for result in mdl.get_topic_words(topic[0], top_n=5)]
    print(f"Topic {topic[0]}: {words} ({topic[1]:.4f})")

The `get_topics()` function returns the topic distribution for each document. The first number in each tuple is the topic number, and the second number is the proportion of that topic in the document. Here I've done some additional processing to make it easier to read:

```
Abstract:
A variety of experiments have demonstrated that people's choices among gambles differ according to
whether the gamble....

Top three topics:
Topic 18: ['use', 'review', 'choice', 'case', 'gamble'] (0.7377)
Topic 14: ['time', 'organizational', 'factor', 'result', 'test'] (0.0955)
Topic 1: ['candidate', 'woman', 'commitment', 'overqualification', 'effectuation'] (0.0652)
```

Here you can see that Topic 18 is the most prevalent in this document, followed by Topic 14 and Topic 1. This can be useful for understanding what topics are present in the corpus and how they are distributed across the documents. Technically, every document is associated with every topic, but some topics are more prevalent in some documents than others. In this case, the drop-off is pretty steep - the top topic is 74% of the document, the second topic is 10%, and the third topic is 7%. So we can be reasonably confident the document is mostly about Topic 18.

We can also have the model 'code' documents it hasn't already seen for us. This can be useful for understanding what topics are present in new documents. Go ahead and try it with an abstract of your own choosing

In [ ]:
unseen_abstract = ""
tokenized_abstract = preprocess(unseen_abstract)
doc = mdl.make_doc(tokenized_abstract)
result = mdl.infer(doc)
print("Unseen abstract:")
print(textwrap.fill(unseen_abstract, width=100))
print("\nTopic distribution over the document:")
topics = result[0]
print(topics)

Here you can see what I mean by each document being associated with every topic. The model has assigned a percentage to each topic. This can be useful for understanding what topics are present in the document and how they are distributed across the topics.

Done...